In [21]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import io
data=pd.read_csv('../../../raw_data/tripcsvbame.csv')

In [23]:
from londonbss.params import *

In [24]:
from londonbss.ml_logic.data import load_data_to_bq

In [25]:
# data['usage-stats/'][:2]

In [26]:
# dat=data['usage-stats/'][:2]
# for item in dat:
#     item=('../../'+(item)).strip()
#     print(item)

In [27]:
# dat

In [28]:
dat=data['usage-stats/'][:2]
frames=[]
for x in dat:
    frames.append(pd.read_csv('../../../raw_data/'+x[1:]))

result = pd.concat(frames)
result.columns

Index(['Rental Id', 'Duration', 'Bike Id', 'End Date', 'EndStation Id',
       'EndStation Name', 'Start Date', 'StartStation Id',
       'StartStation Name'],
      dtype='object')

In [29]:
date_format = '%d/%m/%Y %H:%M:%S'
result['End Date']=pd.to_datetime(result['End Date'],format='mixed')
result['Start Date']=pd.to_datetime(result['Start Date'],format='mixed')
result

,Rental Id,Duration,Bike Id,End Date,EndStation Id,EndStation Name,Start Date,StartStation Id,StartStation Name
0,50754225,240,11834,2016-10-01 00:04:00,383.0,"Frith Street, Soho",2016-10-01 00:00:00,18,"Drury Lane, Covent Garden"
1,50754226,300,9648,2016-10-01 00:05:00,719.0,"Victoria Park Road, Hackney Central",2016-10-01 00:00:00,479,"Pott Street, Bethnal Green"
2,50754227,1200,10689,2016-10-01 00:20:00,272.0,"Baylis Road, Waterloo",2016-10-01 00:00:00,425,"Harrington Square 2, Camden Town"
3,50754228,780,8593,2016-10-01 00:14:00,471.0,"Hewison Street, Old Ford",2016-10-01 00:01:00,487,"Canton Street, Poplar"
4,50754229,600,8619,2016-10-01 00:11:00,399.0,"Brick Lane Market, Shoreditch",2016-10-01 00:01:00,501,"Cephas Street, Bethnal Green"
...,...,...,...,...,...,...,...,...,...
292694,51360634,600,10400,2016-07-02 00:07:00,48.0,"Godliman Street, St. Paul's",2016-06-02 23:57:00,388,"Southampton Street, Strand"
292695,51360635,240,2320,2016-07-02 00:01:00,574.0,"Eagle Wharf Road, Hoxton",2016-06-02 23:57:00,73,"Old Street Station, St. Luke's"
292696,51360636,720,11143,2016-07-02 00:09:00,535.0,"Gloucester Avenue, Camden Town",2016-06-02 23:57:00,109,"Soho Square , Soho"
292697,51360637,900,6848,2016-07-02 00:14:00,732.0,"Duke Street Hill, London Bridge",2016-06-02 23:59:00,443,"Philpot Street, Whitechapel"


In [30]:
result=result.dropna()
new_result=result.set_index('Start Date').groupby([pd.Grouper(freq='h'), 'StartStation Name', 'EndStation Name']).count()
new_result

Rental Id  \
Start Date          StartStation Name                 EndStation Name                                          
2016-01-02 00:00:00 Abbey Orchard Street, Westminster Eccleston Place, Victoria                            2   
                    Abingdon Villas, Kensington       Gloucester Road (Central), South Kensington          1   
                    Aintree Street, Fulham            Imperial Road, Sands End                             1   
                    BBC White City, White City        Crisp Road, Hammersmith                              2   
                    Baldwin Street, St. Luke's        Queen Mary's, Mile End                               1   
...                                                                                                      ...   
2016-12-01 23:00:00 Windsor Terrace, Hoxton           Leonard Circus , Shoreditch                          1   
                    Winsland Street, Paddington       Turquoise Island, Notting Hill                       1   
                    Woodstock Street, Mayfair         Murray Grove , Hoxton                                1   
                    Wormwood Street, Liverpool Street Bank of England Museum, Bank                         1   
                    Wright's Lane, Kensington         Danvers Street, West Chelsea                         1   

                                                                                                   Duration  \
Start Date          StartStation Name                 EndStation Name                                         
2016-01-02 00:00:00 Abbey Orchard Street, Westminster Eccleston Place, Victoria                           2   
                    Abingdon Villas, Kensington       Gloucester Road (Central), South Kensington         1   
                    Aintree Street, Fulham            Imperial Road, Sands End                            1   
                    BBC White City, White City        Crisp Road, Hammersmith                             2   
                    Baldwin Street, St. Luke's        Queen Mary's, Mile End                              1   
...                                                                                                     ...   
2016-12-01 23:00:00 Windsor Terrace, Hoxton           Leonard Circus , Shoreditch                         1   
                    Winsland Street, Paddington       Turquoise Island, Notting Hill                      1   
                    Woodstock Street, Mayfair         Murray Grove , Hoxton                               1   
                    Wormwood Street, Liverpool Street Bank of England Museum, Bank                        1   
                    Wright's Lane, Kensington         Danvers Street, West Chelsea                        1   

                                                                                                   Bike Id  \
Start Date          StartStation Name                 EndStation Name                                        
2016-01-02 00:00:00 Abbey Orchard Street, Westminster Eccleston Place, Victoria                          2   
                    Abingdon Villas, Kensington       Gloucester Road (Central), South Kensington        1   
                    Aintree Street, Fulham            Imperial Road, Sands End                           1   
                    BBC White City, White City        Crisp Road, Hammersmith                            2   
                    Baldwin Street, St. Luke's        Queen Mary's, Mile End                             1   
...                                                                                                    ...   
2016-12-01 23:00:00 Windsor Terrace, Hoxton           Leonard Circus , Shoreditch                        1   
                    Winsland Street, Paddington       Turquoise Island, Notting Hill                     1   
                    Woodstock Street, Mayfair         Murray Grove , Hoxton                       

In [31]:
new_result['No of trips']=new_result['Duration']
new_result.drop(columns=['Rental Id'],inplace=True)
new_result.drop(columns=['Duration'],inplace=True)
new_result.drop(columns=['Bike Id'],inplace=True)
new_result.drop(columns=['End Date'],inplace=True)
new_result.drop(columns=['EndStation Id'],inplace=True)
new_result.drop(columns=['StartStation Id'],inplace=True)
new_result

No of trips
Start Date          StartStation Name                 EndStation Name                                         
2016-01-02 00:00:00 Abbey Orchard Street, Westminster Eccleston Place, Victoria                              2
                    Abingdon Villas, Kensington       Gloucester Road (Central), South Kensington            1
                    Aintree Street, Fulham            Imperial Road, Sands End                               1
                    BBC White City, White City        Crisp Road, Hammersmith                                2
                    Baldwin Street, St. Luke's        Queen Mary's, Mile End                                 1
...                                                                                                        ...
2016-12-01 23:00:00 Windsor Terrace, Hoxton           Leonard Circus , Shoreditch                            1
                    Winsland Street, Paddington       Turquoise Island, Notting Hill                         1
                    Woodstock Street, Mayfair         Murray Grove , Hoxton                                  1
                    Wormwood Street, Liverpool Street Bank of England Museum, Bank                           1
                    Wright's Lane, Kensington         Danvers Street, West Chelsea                           1

[517926 rows x 1 columns]

In [32]:
df = new_result.reset_index()

In [33]:
df

,Start Date,StartStation Name,EndStation Name,No of trips
0,2016-01-02 00:00:00,"Abbey Orchard Street, Westminster","Eccleston Place, Victoria",2
1,2016-01-02 00:00:00,"Abingdon Villas, Kensington","Gloucester Road (Central), South Kensington",1
2,2016-01-02 00:00:00,"Aintree Street, Fulham","Imperial Road, Sands End",1
3,2016-01-02 00:00:00,"BBC White City, White City","Crisp Road, Hammersmith",2
4,2016-01-02 00:00:00,"Baldwin Street, St. Luke's","Queen Mary's, Mile End",1
...,...,...,...,...
517921,2016-12-01 23:00:00,"Windsor Terrace, Hoxton","Leonard Circus , Shoreditch",1
517922,2016-12-01 23:00:00,"Winsland Street, Paddington","Turquoise Island, Notting Hill",1
517923,2016-12-01 23:00:00,"Woodstock Street, Mayfair","Murray Grove , Hoxton",1
517924,2016-12-01 23:00:00,"Wormwood Street, Liverpool Street","Bank of England Museum, Bank",1


In [44]:
df.columns

Index(['____Start Date', '____StartStation Name', '____EndStation Name',
       '____No of trips'],
      dtype='object')

In [49]:
new_names={}
for column_name in df.columns:
    new_names[column_name]=column_name.replace('_','').lstrip().replace(' ','')
new_names

{'____Start Date': 'StartDate',
 '____StartStation Name': 'StartStationName',
 '____EndStation Name': 'EndStationName',
 '____No of trips': 'Nooftrips'}

In [50]:
df=df.rename(columns=new_names)
df

,StartDate,StartStationName,EndStationName,Nooftrips
0,2016-01-02 00:00:00,"Abbey Orchard Street, Westminster","Eccleston Place, Victoria",2
1,2016-01-02 00:00:00,"Abingdon Villas, Kensington","Gloucester Road (Central), South Kensington",1
2,2016-01-02 00:00:00,"Aintree Street, Fulham","Imperial Road, Sands End",1
3,2016-01-02 00:00:00,"BBC White City, White City","Crisp Road, Hammersmith",2
4,2016-01-02 00:00:00,"Baldwin Street, St. Luke's","Queen Mary's, Mile End",1
...,...,...,...,...
517921,2016-12-01 23:00:00,"Windsor Terrace, Hoxton","Leonard Circus , Shoreditch",1
517922,2016-12-01 23:00:00,"Winsland Street, Paddington","Turquoise Island, Notting Hill",1
517923,2016-12-01 23:00:00,"Woodstock Street, Mayfair","Murray Grove , Hoxton",1
517924,2016-12-01 23:00:00,"Wormwood Street, Liverpool Street","Bank of England Museum, Bank",1


In [51]:
load_data_to_bq(df,
        gcp_project=GCP_PROJECT,
        bq_dataset=BQ_DATASET,
        table=f'processed_{DATA_SIZE}',
        truncate=False
    )


Save data to BigQuery @ wagon-bootcamp-396316.londonbss.processed_1k...:
✅ Data saved to bigquery, with shape (517926, 4)


LoadJob<project=wagon-bootcamp-396316, location=EU, id=b101b2fc-e092-4572-a07b-47d6bb76ccba>